<a href="https://colab.research.google.com/github/prasannachiratla/Fmml2024/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

Answer 1
Increasing the percentage of the validation set:

Nearest Neighbor: Accuracy generally decreases. A larger validation set means a smaller training set. With less data for training, the model may not generalize as well, leading to lower accuracy on the validation set.
Random Classifier: Accuracy remains largely unaffected. Since this classifier assigns labels randomly, the size of the validation set doesn't significantly impact its performance.
Reducing the percentage of the validation set:

Nearest Neighbor: Accuracy generally increases. A smaller validation set leads to a larger training set, potentially improving model generalization and validation accuracy. However, a very small validation set may not accurately reflect the model's performance on unseen data.
Random Classifier: Accuracy remains largely unaffected, due to the random nature of its predictions.
In summary: For Nearest Neighbor, a good balance is needed. A larger training set helps, but the validation set needs to be sufficiently large to provide a reliable estimate of the model's performance on unseen data. For the Random Classifier, the size of the validation set has minimal impact due to its inherent randomness.

Answer 2
The size of the training and validation sets plays a crucial role in predicting test set accuracy:

Larger training set: Generally leads to better model generalization, as the model learns from more data, potentially resulting in more accurate predictions on unseen data (test set).

Larger validation set: Provides a more reliable estimate of the model's performance on unseen data. A small validation set may not accurately represent the true performance due to random variations in the data.

The balance: A good balance between the training and validation set sizes is crucial. If the training set is too small, the model may not generalize well. If the validation set is too small, the accuracy estimate might be unreliable.

In the context of your code: The split function controls this balance. Experiment with different split percentages to observe the effect on validation and test accuracy.



#Answer 3
A common split is 80% for training and 20% for validation. However, the ideal percentage depends on the dataset size and complexity:

Larger datasets: Allow for a smaller validation set percentage (e.g., 10% or even 5%), as the larger training set still provides sufficient data for model learning.

Smaller datasets: May require a larger validation set percentage (e.g., 25% or 30%) to ensure a more reliable performance estimate.

Experimentation: It's best to experiment with different split percentages and observe their effect on validation and test accuracy to determine the optimal balance for your specific dataset and model.

In your code: You can modify the splitpercent argument in the split function to experiment with different split ratios and evaluate their impact on model performance.

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 1-Nearest Neighbor
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(alltraindata, alltrainlabel)
pred1 = knn1.predict(testdata)
accuracy1 = accuracy_score(testlabel, pred1)
print(f"Accuracy of 1-NN: {accuracy1 * 100:.2f}%")

# 3-Nearest Neighbor
knn3 = KNeighborsClassifier(n_neighbors=3)
knn3.fit(alltraindata, alltrainlabel)
pred3 = knn3.predict(testdata)
accuracy3 = accuracy_score(testlabel, pred3)
print(f"Accuracy of 3-NN: {accuracy3 * 100:.2f}%")

Accuracy of 1-NN: 34.92%
Accuracy of 3-NN: 36.05%


This code snippet first defines a 1-Nearest Neighbor classifier using KNeighborsClassifier(n_neighbors=1), fits it to the training data, predicts on the test data, and prints the accuracy. It then repeats the process for a 3-Nearest Neighbor classifier using KNeighborsClassifier(n_neighbors=3).

Execute the code yourself to see the output and compare the accuracies. You'll likely observe that the 3-NN model performs differently compared to the 1-NN model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


#Answer 1
Yes, averaging the validation accuracy across multiple splits generally gives more consistent results. This technique, known as cross-validation, helps to reduce the influence of any particular random split on the estimated performance. By averaging the results across multiple splits, you obtain a more robust and reliable estimate of how well your model is likely to perform on unseen data.

#Answer 2
Yes, averaging validation accuracy across multiple splits (cross-validation) often provides a more accurate estimate of test accuracy compared to a single validation split.

Here's why:

Reduced bias: A single split may result in a validation set that is not entirely representative of the overall data distribution. Cross-validation helps mitigate this bias by using different subsets of the data for validation.

Variance reduction: Averaging results across multiple splits reduces the variance in the estimated accuracy, leading to a more stable and reliable estimate of the model's generalization performance.

Better utilization of data: Cross-validation allows you to use more of your data for both training and validation, leading to a more comprehensive evaluation of the model.

#Answer 3
Generally, a higher number of iterations in cross-validation leads to a more accurate and stable estimate of model performance. Here's why:

More data used: Each iteration in cross-validation involves a different split of the data. With more iterations, the model is trained and evaluated on a larger portion of the data, leading to a more comprehensive assessment.

Reduced noise: Averaging results over a larger number of iterations helps to reduce the impact of random variations in the data and provides a more reliable estimate of the average performance.

Computational cost: However, increasing the number of iterations also increases the computational cost of cross-validation. You need to balance the desired accuracy with the available computational resources.

Diminishing returns: The improvement in accuracy with increasing iterations tends to have diminishing returns. After a certain point, adding more iterations might not significantly improve the estimate.

#Answer 4
While increasing iterations in cross-validation can improve the accuracy and stability of the estimate, it cannot fully compensate for a very small training or validation dataset. Here's why:

Limited learning: A very small training dataset restricts the model's ability to learn effectively and generalize to unseen data, regardless of the number of cross-validation iterations.

Unreliable estimate: A very small validation dataset may not adequately represent the overall data distribution, even with multiple iterations. The accuracy estimate might still be unreliable due to the limited sample size.

Strategies for small datasets:

Data augmentation: If applicable, generate synthetic data to increase the training set size.

Simpler models: Choose models with fewer parameters to avoid overfitting on limited data.

Regularization: Use techniques like L1 or L2 regularization to prevent overfitting.

Transfer learning: Leverage pre-trained models on larger datasets to improve performance on your small dataset.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

Number of Splits:

3-NN: Generally, increasing the number of splits in cross-validation (e.g., using k-fold cross-validation with a larger k) leads to a more stable and accurate estimate of the 3-NN classifier's performance. This is because the model is trained and evaluated on more diverse subsets of the data.

Comparison with 1-NN: Both 1-NN and 3-NN benefit from increased splits, but the effect might be more pronounced for 3-NN as it is more sensitive to variations in the training data due to considering multiple neighbors.

Split Size:

3-NN: The split size (ratio of training to validation data) influences the bias-variance trade-off. Larger training sets generally lead to better generalization, while larger validation sets provide more reliable performance estimates.
A good balance is needed.
Comparison with 1-NN: 1-NN might be more sensitive to smaller training sets as it relies heavily on individual data points. 3-NN might be slightly more robust due to averaging the influence of multiple neighbors.
Experimentation:

To get a precise understanding, experiment with different numbers of splits and split sizes using your code. Modify the iterations and splitpercent arguments in the AverageAccuracy function and observe the changes in average validation accuracy. Compare the results for both 1-NN and 3-NN to see how they are affected by these factors.